In [ ]:
from nbdev import *
%nbdev_default_export test

Cells will be exported to nbdev.test,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
from nbdev.imports import *
from nbdev.sync import *
from nbdev.export import *
from nbdev.export import _mk_flag_re

from nbconvert.preprocessors import ExecutePreprocessor

# Extract tests

> The functions that grab the cells containing tests (filtering with potential flags) and execute them

Everything that is not an exported cell is considered a test, so you should make sure your notebooks can all run smoothly (and fast) if you want to use this functionality as the CLI. You can mark some cells with special flags (like slow) to make sure they are only executed when you authorize it. Those flags should be configured in your `settings.ini` (separated by a `|` if you have several of them). You can also apply flags to one entire notebook by using the `all` option of the test flag, e.g. `%nbdev_slow_test all`, in code cells.

If `tst_flags=slow|fastai2` in `settings.ini`, you can:
- mark slow tests with the `%nbdev_slow_test` flag
- mark tests that depend on fastai2 with the `%nbdev_fastai2_test` flag.

## Detect flags

The following functions detect the cells that should be excluded from the tests (unless their special flag is passed).

In [ ]:
%nbdev_export
_re_all_flag = ReTstFlags(True)

In [ ]:
%nbdev_export
def get_all_flags(cells):
    "Check for all test flags in `cells`"
    if len(Config().get('tst_flags',''))==0: return []
    result = []
    for cell in cells:
        if cell['cell_type'] == 'code': result.extend(_re_all_flag.findall(cell['source']))
    return set(result)

In [ ]:
nb = read_nb("04_test.ipynb")
assert get_all_flags(nb['cells']) == set()

In [ ]:
%nbdev_hide
tst_flags_bck=Config().get('tst_flags')
try:
    Config()['tst_flags'] = 'fastai2|vslow'
    if hasattr(_re_all_flag, '_re'): del _re_all_flag._re
    cells = [{'cell_type': cell_type, 'source': source} for cell_type, source in [
        ('code', '# export\nfrom local.core import *'), 
        ('markdown', '# title of some kind'), 
        ('code', '# all_vslow \n# all_fastai2'),
        ('code', '%nbdev_vslow_test all\n# all_fastai2'),
        ('code', '%nbdev_vslow_test all '),
        ('code', '# all_fastai2'),
        ('code', '%nbdev_fastai2_test  all\n')]]
    for i in range(3):
        test_eq(set(['vslow','fastai2']), get_all_flags(cells))
        cells.pop(2)
    for i in range(2):
        test_eq(set(['fastai2']), get_all_flags(cells))
        cells.pop(2)
    test_eq(set(), get_all_flags(cells))
finally:
    Config()['tst_flags'] = tst_flags_bck
    del _re_all_flag._re

In [ ]:
%nbdev_export
_re_flags = ReTstFlags(False)

In [ ]:
%nbdev_export
def get_cell_flags(cell):
    "Check for any special test flag in `cell`"
    if cell['cell_type'] != 'code' or len(Config().get('tst_flags',''))==0: return []
    return _re_flags.findall(cell['source'])

In [ ]:
test_eq(get_cell_flags({'cell_type': 'code', 'source': "%nbdev_hide\n%nbdev_fastai2_test\n"}), ['fastai2'])
test_eq(get_cell_flags({'cell_type': 'code', 'source': "%nbdev_hide\n"}), [])

In [ ]:
%nbdev_hide
for expected, flag in [(['fastai2'], 'fastai2'), ([], 'vslow')]:
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n# {flag}\n")))
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n%nbdev_{flag}_test\n")))
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"# {flag}\n#hide\n")))
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"%nbdev_{flag}_test\n#hide\n")))
    test_eq([], get_cell_flags(nbformat.v4.new_code_cell("#hide\n")))
    test_eq([], get_cell_flags(nbformat.v4.new_code_cell(f"# all_{flag}")))
    test_eq([], get_cell_flags(nbformat.v4.new_code_cell(f"%nbdev_{flag}_test all")))
tst_flags_bck=Config().get('tst_flags')
try:
    Config()['tst_flags'] = 'fastai2|vslow'
    del _re_flags._re
    test_eq(['vslow'], get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n# vslow\n")))
    test_eq(['vslow'], get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n%nbdev_vslow_test\n")))
    test_eq(['vslow', 'fastai2'], get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n# vslow\n# fastai2")))
    test_eq(['fastai2', 'vslow'], get_cell_flags(nbformat.v4.new_code_cell(f"%nbdev_fastai2_test\n%nbdev_vslow_test")))
finally:
    Config()['tst_flags'] = tst_flags_bck
    del _re_flags._re

## Testing a notebook

In [ ]:
%nbdev_export
class NoExportPreprocessor(ExecutePreprocessor):
    "An `ExecutePreprocessor` that executes cells that don't have a flag in `flags`"
    def __init__(self, flags, **kwargs):
        self.flags = flags
        super().__init__(**kwargs)

    def preprocess_cell(self, cell, resources, index):
        if 'source' not in cell or cell['cell_type'] != "code": return cell, resources
        for f in get_cell_flags(cell):
            if f not in self.flags: return cell, resources
        res = super().preprocess_cell(cell, resources, index)
        return res

In [ ]:
%nbdev_export
def test_nb(fn, flags=None):
    "Execute tests in notebook in `fn` with `flags`"
    os.environ["IN_TEST"] = '1'
    if flags is None: flags = []
    try:
        nb = read_nb(fn)
        for f in get_all_flags(nb['cells']):
            if f not in flags: return
        ep = NoExportPreprocessor(flags, timeout=600, kernel_name='python3')
        pnb = nbformat.from_dict(nb)
        ep.preprocess(pnb)
    finally: os.environ.pop("IN_TEST")

## Export-

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 08_flag_tests.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
Converted tutorial.ipynb.
